In [1]:
from bokeh.plotting import figure, show, ColumnDataSource
from bokeh.io import output_notebook
import bokeh.palettes as bp
import pandas as pd
import collections
import math
import itertools
import operator

In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
df = pd.read_csv('all_data.txt')
# columns are movies, rows are users

In [4]:
# get rid of whitespace and characters that cause issues with pandas column names
df.columns = [i.replace(' ','_') for i in df.columns]
df.columns = [i.replace('(','') for i in df.columns]
df.columns = [i.replace(')','') for i in df.columns]
df.columns = [i.replace(':','') for i in df.columns]
movies_with_rt = df.columns[1:]

In [5]:
# go over every cell in dataframe to get all guesses
Point = collections.namedtuple('Point', 'movie user guess')
Movie = collections.namedtuple('Movie', 'name rt_value')
points = []
for index, row in df.iterrows():
    user = row.user
    for movie in movies_with_rt:
        guess = row[movie]
        if not math.isnan(guess):
            points.append(Point(movie, user, guess))
            
# build dict to convert movie column to name and rt_value
movie_info = {}
for movie in movies_with_rt:
    movie_name = ' '.join(movie.split('_')[:-1])
    rt_value = int(movie.split('_')[-1])
    movie_info[movie] = Movie(movie_name, rt_value)
    
# get list of movies sorted by rt_value so can color them differently    
movie_rt_value = [(movie, info.rt_value) for movie, info in movie_info.items()]
sorted_movies = [i[0] for i in sorted(movie_rt_value, key=operator.itemgetter(1))]

# get a color for each movie, cycling through a list of colors
colors = list(itertools.islice(itertools.cycle(bp.d3['Category10'][10]), 0, len(movies_with_rt)))
movie_to_color = {movie: color for movie, color in zip(sorted_movies, colors)}

In [6]:
rt_values = [movie_info[i.movie].rt_value for i in points]
guesses = [i.guess for i in points]
movies = [movie_info[i.movie].name for i in points]
users = [i.user for i in points]
colors = [movie_to_color[i.movie] for i in points]

In [7]:
source = ColumnDataSource(data=dict(
    x=guesses,
    y=rt_values,
    movies=movies,
    users=users,
    color=colors
))

TOOLTIPS = [
    ('user', '@users'),
    ('movie', '@movies'),
    ('RT', '@y'),
    ('guess', '@x')
]

p = figure(plot_width=800, plot_height=800, tooltips=TOOLTIPS,
           title="How well can Redditors predict Rotten Tomato scores")
p.xaxis.axis_label = 'Guess'
p.yaxis.axis_label = 'Actual'
p.circle('x', 'y', color='color',size=10, source=source, fill_alpha=0.2, )
p.line([0,100],[0,100])

bokeh.models.renderers.GlyphRenderer(
    id='bba470f8-42eb-4f74-b632-15d7bae73247',
    data_source=bokeh.models.sources.ColumnDataSource(
        id='69933d69-5408-49cb-a971-9d041306702b',
        callback=None,
        data={'x': [0, 100], 'y': [0, 100]},
        js_event_callbacks={},
        js_property_callbacks={},
        name=None,
        selected=bokeh.models.selections.Selection(
            id='a1d4f058-b271-444d-85aa-d6e5feb57a85',
            indices=[],
            js_event_callbacks={},
            js_property_callbacks={},
            line_indices=[],
            multiline_indices={},
            name=None,
            subscribed_events=[],
            tags=[]),
        selection_policy=bokeh.models.selections.UnionRenderers(
            id='0aa446e2-d7a0-4ff4-941b-6a48277df442',
            js_event_callbacks={},
            js_property_callbacks={},
            name=None,
            subscribed_events=[],
            tags=[]),
        subscribed_events=[],
        tags=[]),
    glyph=bokeh.models.glyphs.Line(
        id='9b56e49a-852e-44c2-af24-414008f476d8',
        js_event_callbacks={},
        js_property_callbacks={},
        line_alpha=1.0,
        line_cap='butt',
        line_color='#1f77b4',
        line_dash=[],
        line_dash_offset=0,
        line_join='bevel',
        line_width=1,
        name=None,
        subscribed_events=[],
        tags=[],
        x={'field': 'x'},
        y={'field': 'y'}),
    hover_glyph=None,
    js_event_callbacks={},
    js_property_callbacks={},
    level='glyph',
    muted=False,
    muted_glyph=None,
    name=None,
    nonselection_glyph=bokeh.models.glyphs.Line(
        id='2e533fac-02fc-4148-a010-063f57652c76',
        js_event_callbacks={},
        js_property_callbacks={},
        line_alpha=0.1,
        line_cap='butt',
        line_color='#1f77b4',
        line_dash=[],
        line_dash_offset=0,
        line_join='bevel',
        line_width=1,
        name=None,
        subscribed_events=[],
        tags=[],
        x={'field': 'x'},
        y={'field': 'y'}),
    selection_glyph=None,
    subscribed_events=[],
    tags=[],
    view=bokeh.models.sources.CDSView(
        id='5190e9dd-2281-4bf6-889a-20ccd8b4335c',
        filters=[],
        js_event_callbacks={},
        js_property_callbacks={},
        name=None,
        source=bokeh.models.sources.ColumnDataSource(
            id='69933d69-5408-49cb-a971-9d041306702b',
            callback=None,
            data={'x': [0, 100], 'y': [0, 100]},
            js_event_callbacks={},
            js_property_callbacks={},
            name=None,
            selected=bokeh.models.selections.Selection(
                id='a1d4f058-b271-444d-85aa-d6e5feb57a85',
                indices=[],
                js_event_callbacks={},
                js_property_callbacks={},
                line_indices=[],
                multiline_indices={},
                name=None,
                subscribed_events=[],
                tags=[]),
            selection_policy=bokeh.models.selections.UnionRenderers(
                id='0aa446e2-d7a0-4ff4-941b-6a48277df442',
                js_event_callbacks={},
                js_property_callbacks={},
                name=None,
                subscribed_events=[],
                tags=[]),
            subscribed_events=[],
            tags=[]),
        subscribed_events=[],
        tags=[]),
    visible=True,
    x_range_name='default',
    y_range_name='default')

In [8]:
show(p)

In [9]:
from IPython.display import HTML

In [10]:
HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
The raw code for this IPython notebook is by default hidden for easier reading.
To toggle on/off the raw code, click <a href="javascript:code_toggle()">here</a>.''')